In [6]:
%pip install numpy pandas 


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import pandas as pd
import numpy as np
df = pd.read_csv("C://Users/Emrehan/Desktop/Emrehan Simsek/veri çekme selenium/datasets/amazon_products.csv")
df.head()

,Product Name,Price,Image URL,Link
0,Lenovo IdeaPad Slim 3 15IAH8 Dizüstü Bilgisaya...,NaN,https://m.media-amazon.com/images/I/61wH5UlX-M...,https://www.amazon.com.tr/Lenovo-Bilgisayar-i5...
1,ASUS Vivobook 15 - X1504ZA-NJ1266 15.6 inç FHD...,NaN,https://m.media-amazon.com/images/I/51dHk31mrQ...,https://www.amazon.com.tr/ASUS-Vivobook-15-X15...
2,"Lenovo IdeaPad 3 Dizüstü Bilgisayar, Intel Cor...",NaN,https://m.media-amazon.com/images/I/61DQ67Cr2H...,https://www.amazon.com.tr/Lenovo-IdeaPad-Bilgi...
3,ASUS Vivobook 15 - X1504ZA-NJ566 15.6 inç FHD ...,NaN,https://m.media-amazon.com/images/I/51zyo-VeNS...,https://www.amazon.com.tr/ASUS-Vivobook-15-X15...
4,Asus Vivobook 15 X1504ZA-NJ180 Intel i5-1235U ...,NaN,https://m.media-amazon.com/images/I/514DRQG0T7...,https://www.amazon.com.tr/Vivobook-X1504ZA-NJ1...


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405 entries, 0 to 404
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Product Name  405 non-null    object 
 1   Price         0 non-null      float64
 2   Image URL     405 non-null    object 
 3   Link          405 non-null    object 
dtypes: float64(1), object(3)
memory usage: 12.8+ KB


In [9]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

# Geckodriver'ın yolu
driver_path = "C:\\Users\\EMREHAN\\PycharmProjects\\geckodriver.exe"
service = Service(driver_path)

# Firefox WebDriver'ı headless modda başlat
options = webdriver.FirefoxOptions()
options.add_argument("--headless")
# (İsteğe bağlı) User-Agent ayarını değiştirmek:
# options.set_preference("general.useragent.override", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) ...")

driver = webdriver.Firefox(service=service, options=options)

# CSV dosyasını oku
csv_file_path = "C://Users/EMREHAN/Desktop/Emrehan Simsek/veri çekme selenium/datasets/amazon_products.csv"
df = pd.read_csv(csv_file_path)

# Eğer "Price" kolonu yoksa, oluştur
if "Price" not in df.columns:
    df["Price"] = "N/A"


for index, row in df.iterrows():
    product_url = row["Link"]
    try:
        driver.get(product_url)
        price = "N/A"  # Varsayılan fiyat değeri
        
        # 1. Yöntem: Hiyerarşik yapıdaki fiyat bilgisini çekmeye çalışalım.
        try:
            price_container = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((
                    By.CSS_SELECTOR,
                    "span.a-price.aok-align-center.reinventPricePriceToPayMargin.priceToPay"
                ))
            )
            # Fiyat bileşenlerini alalım
            whole = price_container.find_element(By.CSS_SELECTOR, "span.a-price-whole").text.strip()
            decimal = price_container.find_element(By.CSS_SELECTOR, "span.a-price-decimal").text.strip()
            fraction = price_container.find_element(By.CSS_SELECTOR, "span.a-price-fraction").text.strip()
            symbol = price_container.find_element(By.CSS_SELECTOR, "span.a-price-symbol").text.strip()
            price = f"{whole}{decimal}{symbol}"
        except Exception as hierarchical_exception:
            # Hiyerarşik yapıdan alamazsak diğer yöntemlere geçelim.
            
            # 2. Yöntem: 'span.a-offscreen' elementini bekleyelim.
            try:
                price_element = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "span.a-offscreen"))
                )
                price = price_element.text.strip()
            except Exception as e:
                # 3. Yöntem: 'a-price-whole' ve 'a-price-fraction' kombinasyonunu deneyelim.
                try:
                    whole_element = WebDriverWait(driver, 3).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, "span.a-price-whole"))
                    )
                    fraction_element = driver.find_element(By.CSS_SELECTOR, "span.a-price-fraction")
                    price = f"{whole_element.text.strip()},{fraction_element.text.strip()} TL"
                except Exception as e2:
                    # 4. Yöntem: Son çare olarak BeautifulSoup ile sayfa kaynak kodunu kontrol edelim.
                    time.sleep(2)  # Ekstra bekleme, sayfanın tam yüklenmesi için
                    soup = BeautifulSoup(driver.page_source, "html.parser")
                    price_tag = soup.find("span", class_="a-offscreen")
                    if price_tag:
                        price = price_tag.text.strip()
                    else:
                        whole_tag = soup.find("span", class_="a-price-whole")
                        fraction_tag = soup.find("span", class_="a-price-fraction")
                        if whole_tag and fraction_tag:
                            price = f"{whole_tag.text.strip()},{fraction_tag.text.strip()} TL"
                        else:
                            price = "N/A"
        
        # Fiyat bilgisini DataFrame'e kaydet
        df.at[index, "Price"] = price
        print(f"{index + 1}. ürün fiyatı alındı: {price}")
    
    except Exception as main_exception:
        print(f"Fiyat çekilemedi ({product_url}): {main_exception}")
        df.at[index, "Price"] = "N/A"

# Tarayıcıyı kapat
driver.quit()
print("Fiyat bilgileri güncellendi ve CSV'ye kaydedildi.")

# Yarin bunu tüm dataset icin yap. Dakikada 10 tane güncelliyor, tahmini 40 dakika sürer.

C:\Users\EMREHAN\AppData\Local\Temp\ipykernel_23604\3633552196.py:84: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '14.999TL' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, "Price"] = price


1. ürün fiyatı alındı: 14.999TL
2. ürün fiyatı alındı: 11.659TL
3. ürün fiyatı alındı: 10.999TL


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import concurrent.futures

# Geckodriver'ın yolu
driver_path = "C:\\Users\\EMREHAN\\PycharmProjects\\geckodriver.exe"

def fetch_price(product_url):
    """
    Belirtilen ürün URL'sinden fiyat bilgisini çekip döndürür.
    Her çağrıda yeni bir Selenium driver oluşturulur.
    """
    price = "N/A"  # Varsayılan fiyat değeri
    options = webdriver.FirefoxOptions()
    options.add_argument("--headless")
    # (İsteğe bağlı) User-Agent ayarı yapılabilir:
    # options.set_preference("general.useragent.override", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) ...")
    service = Service(driver_path)
    driver = webdriver.Firefox(service=service, options=options)

    try:
        driver.get(product_url)

        # 1. Yöntem: Hiyerarşik yapıdaki fiyat bilgisini çekmeye çalışalım.
        try:
            price_container = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((
                    By.CSS_SELECTOR,
                    "span.a-price.aok-align-center.reinventPricePriceToPayMargin.priceToPay"
                ))
            )
            # Fiyat bileşenlerini alalım
            whole = price_container.find_element(By.CSS_SELECTOR, "span.a-price-whole").text.strip()
            decimal = price_container.find_element(By.CSS_SELECTOR, "span.a-price-decimal").text.strip()
            try:
                fraction = price_container.find_element(By.CSS_SELECTOR, "span.a-price-fraction").text.strip()
                price = f"{whole}{decimal}{fraction}"
            except Exception:
                price = f"{whole}{decimal}"
            symbol = price_container.find_element(By.CSS_SELECTOR, "span.a-price-symbol").text.strip()
            price = f"{price} {symbol}"

        except Exception as hierarchical_exception:
            # 2. Yöntem: 'span.a-offscreen' elementini bekleyelim.
            try:
                price_element = WebDriverWait(driver, 3).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "span.a-offscreen"))
                )
                price = price_element.text.strip()
            except Exception as e:
                # 3. Yöntem: 'a-price-whole' ve 'a-price-fraction' kombinasyonunu deneyelim.
                try:
                    whole_element = WebDriverWait(driver, 3).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, "span.a-price-whole"))
                    )
                    fraction_element = driver.find_element(By.CSS_SELECTOR, "span.a-price-fraction")
                    price = f"{whole_element.text.strip()},{fraction_element.text.strip()} TL"
                except Exception as e2:
                    # 4. Yöntem: Son çare olarak BeautifulSoup ile sayfa kaynak kodunu kontrol edelim.
                    time.sleep(2)  # Ekstra bekleme
                    soup = BeautifulSoup(driver.page_source, "html.parser")
                    price_tag = soup.find("span", class_="a-offscreen")
                    if price_tag:
                        price = price_tag.text.strip()
                    else:
                        whole_tag = soup.find("span", class_="a-price-whole")
                        fraction_tag = soup.find("span", class_="a-price-fraction")
                        if whole_tag and fraction_tag:
                            price = f"{whole_tag.text.strip()},{fraction_tag.text.strip()} TL"
                        else:
                            price = "N/A"

    except Exception as main_exception:
        price = "N/A"
    finally:
        driver.quit()

    return price

if __name__ == "__main__":
    # CSV dosyasını oku
    csv_file_path = "C://Users/EMREHAN/Desktop/Emrehan Simsek/veri çekme selenium/datasets/amazon_products.csv"
    df = pd.read_csv(csv_file_path)

    # Eğer "Price" kolonu yoksa, oluştur
    if "Price" not in df.columns:
        df["Price"] = "N/A"

    # Ürün URL'lerini listeye alalım
    urls = df["Link"].tolist()

    # Paralel işlemler için ThreadPoolExecutor kullanıyoruz.
    # max_workers: aynı anda kaç tarayıcı örneği çalıştırılacağını belirler.
    max_workers = 5
    prices = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Her URL için fetch_price fonksiyonunu çalıştırıyoruz.
        future_to_url = {executor.submit(fetch_price, url): url for url in urls}

        for future in concurrent.futures.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                price = future.result()
            except Exception as exc:
                print(f"{url} için hata oluştu: {exc}")
                price = "N/A"
            prices.append(price)
            print(f"URL: {url} => Fiyat: {price}")

    # Elde edilen fiyatları DataFrame'e kaydedelim.
    df["Price"] = prices

    # Güncellenmiş CSV'yi kaydedelim
    output_csv = "C://Users/EMREHAN/Desktop/Emrehan Simsek/veri çekme selenium/datasets/amazon_products_updated.csv"
    df.to_csv(output_csv, index=False)

    print("Fiyat bilgileri güncellendi ve CSV'ye kaydedildi.")


URL: https://www.amazon.com.tr/Lenovo-IdeaPad-Bilgisayar-i3-1215U-82RK011BTX/dp/B0CQTKWH2M/ref=sr_1_3?dib=eyJ2IjoiMSJ9.cAStGyXY4Ttlop_Td9PZXiT8XUwEgMm5EFh6-h95Y1hWj9uUUBoaD8md3A4UjnjuIagUE3vJx3KnUi-JvH8cKk0HTEGAX8kO0Bhtc8QUjYa2l6_zy0ln6kP2qI_6Kxoh-nDmgMQKQl-tP6Q3U9i6Xdo8nrDd85-9hZEJrFyfJIHDVKHvBuVDoc8__xz73XZ515r2uBFMSt5cRM1v22ysRKWFcdViGJQCj0yZA6WUJqAwlS7Qbx81kqBIQYsncmEogGLZXwAk8vb032RdTv_eCvYz9uDT9D7To-w7UHkP2VrMV276jD3bNLuT6iBIwgFyMQ8yLp_hU-b_xy0aYApkCHejwbw8mlO-w_2rrCVVzck.Oyo7D-r7aZUqHTPMtdljGI-kKT6WVuVoCeztVUvTInw&dib_tag=se&qid=1738967976&s=computers&sr=1-3 => Fiyat: 10.99900 TL
URL: https://www.amazon.com.tr/ASUS-Vivobook-15-X1504ZA-NJ1266-Bilgisayar/dp/B08H1WX5MY/ref=sr_1_2?dib=eyJ2IjoiMSJ9.cAStGyXY4Ttlop_Td9PZXiT8XUwEgMm5EFh6-h95Y1hWj9uUUBoaD8md3A4UjnjuIagUE3vJx3KnUi-JvH8cKk0HTEGAX8kO0Bhtc8QUjYa2l6_zy0ln6kP2qI_6Kxoh-nDmgMQKQl-tP6Q3U9i6Xdo8nrDd85-9hZEJrFyfJIHDVKHvBuVDoc8__xz73XZ515r2uBFMSt5cRM1v22ysRKWFcdViGJQCj0yZA6WUJqAwlS7Qbx81kqBIQYsncmEogGLZXwAk8vb032RdTv_eCvYz9uDT9D7To-

In [ ]:
df.head()

,Product Name,Price,Image URL,Link
0,Lenovo IdeaPad Slim 3 15IAH8 Dizüstü Bilgisaya...,10.99900 TL,https://m.media-amazon.com/images/I/61wH5UlX-M...,https://www.amazon.com.tr/Lenovo-Bilgisayar-i5...
1,ASUS Vivobook 15 - X1504ZA-NJ1266 15.6 inç FHD...,11.65900 TL,https://m.media-amazon.com/images/I/51dHk31mrQ...,https://www.amazon.com.tr/ASUS-Vivobook-15-X15...
2,"Lenovo IdeaPad 3 Dizüstü Bilgisayar, Intel Cor...",17.09900 TL,https://m.media-amazon.com/images/I/61DQ67Cr2H...,https://www.amazon.com.tr/Lenovo-IdeaPad-Bilgi...
3,ASUS Vivobook 15 - X1504ZA-NJ566 15.6 inç FHD ...,14.49900 TL,https://m.media-amazon.com/images/I/51zyo-VeNS...,https://www.amazon.com.tr/ASUS-Vivobook-15-X15...
4,Asus Vivobook 15 X1504ZA-NJ180 Intel i5-1235U ...,14.99900 TL,https://m.media-amazon.com/images/I/514DRQG0T7...,https://www.amazon.com.tr/Vivobook-X1504ZA-NJ1...


In [ ]:
df["Price"]

0      10.99900 TL
1      11.65900 TL
2      17.09900 TL
3      14.49900 TL
4      14.99900 TL
          ...     
400               
401    29.34900 TL
402    27.94900 TL
403    51.29900 TL
404    57.89900 TL
Name: Price, Length: 405, dtype: object

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405 entries, 0 to 404
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Product Name  405 non-null    object
 1   Price         405 non-null    object
 2   Image URL     405 non-null    object
 3   Link          405 non-null    object
dtypes: object(4)
memory usage: 12.8+ KB


,Product Name,Price,Image URL,Link
14,Lenovo V15 82YU00QKTX Ryzen5 7520U 8GB 512SSD ...,,https://m.media-amazon.com/images/I/619iKaqHBQ...,https://www.amazon.com.tr/Lenovo-82YU00QKTX-Fr...
37,Lenovo Ideapad Gaming 3 AMD Ryzen 5 5500H 16GB...,,https://m.media-amazon.com/images/I/61PzxoPDzX...,https://www.amazon.com.tr/Lenovo-Ideapad-Ta%C5...
93,"MSI CYBORG 15 A13VF-895XTR Dizüstü Bilgisayar,...",,https://m.media-amazon.com/images/I/71l4M2y8zw...,https://www.amazon.com.tr/MSI-A13VF-895XTR-Diz...
94,MSI NB MODERN 15 B12MO-676XTR I5-1235U 16GB DD...,,https://m.media-amazon.com/images/I/718AiQQtJB...,https://www.amazon.com.tr/MSI-MODERN-B12MO-676...
142,Acer Aspire Lite AL14-31P-C452 Intel N100 8 GB...,,https://m.media-amazon.com/images/I/61h0P70LY7...,https://www.amazon.com.tr/AL14-31P-C452-Ta%C5%...
153,Lenovo LOQ AMD Ryzen 5 8645HS 16GB 1TB SSD RTX...,,https://m.media-amazon.com/images/I/61OLCvNdfQ...,https://www.amazon.com.tr/Lenovo-RTX4050-Ta%C5...
166,Dell Gaming G15 5530 G155530014U Intel Core i7...,,https://m.media-amazon.com/images/I/51k5EahA-2...,https://www.amazon.com.tr/Dell-Gaming-G1555300...
198,"MSI THIN A15 B7VE-076XTR Dizüstü Bilgisayar, R...",,https://m.media-amazon.com/images/I/71Fr-EEsp6...,https://www.amazon.com.tr/MSI-B7VE-076XTR-Diz%...
202,Dell Vostro 3520 N1614PVNB3520U13 i3-1215U 32G...,,https://m.media-amazon.com/images/I/71nbaKV1BN...,https://www.amazon.com.tr/Dell-N1614PVNB3520U1...
209,HP Victus 16-R1001NT 9J205EA i7-14700HX 16GB 1...,,https://m.media-amazon.com/images/I/617T8C2dil...,https://www.amazon.com.tr/HP-16-R1001NT-RTX406...


In [ ]:
df["Link"][37] #21.49900 TL
df["Link"][93] # Fiyat Bilgisi yok, bunu at.
df["Link"][94] #20.89900 TL
df["Link"][142] #9.69900 TL
df["Link"][153] #36.99900 TL
df["Link"][166]#46.40900 TL
df["Link"][198] #Fiyat Bilgisi yok, bunu at.
df["Link"][202] #Fiyat Bilgisi yok, bunu at.
df["Link"][209] #50.39900 TL


https://www.amazon.com.tr/HP-16-R1001NT-RTX4060-8GB-Ta%C5%9F%C4%B1nabilir-Bilgisayar/dp/B0CYTC2Y87/ref=sr_1_189?dib=eyJ2IjoiMSJ9.Z7LY3menxp1_Dm_2Y0IKTxXEhg9jgBIhYmmQ6dyP60sIs2TbxdJKnO9CI-5MQr6Kv1rLdNIYD6yv9ibtT-6vko-jBSZGJHVrmA53ir0DrnkgQrC16gnmina_LVCSYKRqhCMzY5KKPmHJmt6fZPy6MtfSf0EZDmU32y3dNW0ZNPj0Oh14fR3xLgyw4bb9BRCB6bau3YUb5LHeUQawA6QWVp9Y_4VcgKAS7ZpEZVjGpvYs1XCJaCPcoyAl581h3fGE3WZbpCIAMIjdoD6pbktVFf5JB-yCErcyU3Us-iJSXFEpe14d0ZIv8xcWJTFm5IeDFYuLfjERnqWcgqW-VTBe8sveg2c9r2q8aCh5VtDpiQg.67tQ3KSI4k6nN5EG_23C9J0RumI9yjBnAkCYcPreMlw&dib_tag=se&qid=1738968042&s=computers&sr=1-189


In [ ]:
empty_price_laptops.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38 entries, 14 to 400
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Product Name  38 non-null     object
 1   Price         38 non-null     object
 2   Image URL     38 non-null     object
 3   Link          38 non-null     object
dtypes: object(4)
memory usage: 1.5+ KB


In [ ]:
df["Price"][209] = "50.30900 TL"  

In [ ]:
empty_price_laptops = df[df["Price"] == ""]

In [ ]:
empty_price_laptops.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32 entries, 14 to 400
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Product Name  32 non-null     object
 1   Price         32 non-null     object
 2   Image URL     32 non-null     object
 3   Link          32 non-null     object
dtypes: object(4)
memory usage: 1.2+ KB


In [10]:
import re
def extract_asin(url):
    """
    Amazon ürün linkinden ASIN değerini çıkarır.
    
    Args:
        url (str): Amazon ürün linki
        
    Returns:
        str: ASIN değeri veya boş string
    """
    try:
        # /dp/ formatındaki ASIN'leri bul
        pattern = r'/dp/([A-Z0-9]{10})'
        match = re.search(pattern, url)
        
        if match:
            return match.group(1)
        return ''
    except:
        return ''
    
    # Link sütunundan ASIN değerlerini çıkar
df['ASIN'] = df['Link'].apply(extract_asin)
        
    # İstatistikleri hesapla ve göster
total_rows = len(df)
empty_asins = df['ASIN'].str.len().eq(0).sum()
valid_asins = total_rows - empty_asins
        
print(f"Toplam işlenen satır: {total_rows}")
print(f"Geçerli ASIN bulunan: {valid_asins}")
print(f"ASIN bulunamayan: {empty_asins}")
        
      
        
    # İlk birkaç ASIN değerini göster
print("\nİlk 5 ürünün ASIN değerleri:")
for _, row in df.head().iterrows():
    product_name = row['Product Name'][:50] + "..." if len(row['Product Name']) > 50 else row['Product Name']
    print(f"Ürün: {product_name}")
    print(f"ASIN: {row['ASIN']}\n")
            
   
        

Toplam işlenen satır: 405
Geçerli ASIN bulunan: 360
ASIN bulunamayan: 45

İlk 5 ürünün ASIN değerleri:
Ürün: Lenovo IdeaPad Slim 3 15IAH8 Dizüstü Bilgisayar, 1...
ASIN: B0CK84VL9X

Ürün: ASUS Vivobook 15 - X1504ZA-NJ1266 15.6 inç FHD (19...
ASIN: B08H1WX5MY

Ürün: Lenovo IdeaPad 3 Dizüstü Bilgisayar, Intel Core i3...
ASIN: B0CQTKWH2M

Ürün: ASUS Vivobook 15 - X1504ZA-NJ566 15.6 inç FHD (192...
ASIN: B08HLQTYXW

Ürün: Asus Vivobook 15 X1504ZA-NJ180 Intel i5-1235U 16GB...
ASIN: B0DCYYFMYC



In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 405 entries, 0 to 404
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Product Name  405 non-null    object
 1   Price         3 non-null      object
 2   Image URL     405 non-null    object
 3   Link          405 non-null    object
 4   ASIN          405 non-null    object
dtypes: object(5)
memory usage: 15.9+ KB


In [12]:
df = df[df["ASIN"] != ""]
df

,Product Name,Price,Image URL,Link,ASIN
0,Lenovo IdeaPad Slim 3 15IAH8 Dizüstü Bilgisaya...,14.999TL,https://m.media-amazon.com/images/I/61wH5UlX-M...,https://www.amazon.com.tr/Lenovo-Bilgisayar-i5...,B0CK84VL9X
1,ASUS Vivobook 15 - X1504ZA-NJ1266 15.6 inç FHD...,11.659TL,https://m.media-amazon.com/images/I/51dHk31mrQ...,https://www.amazon.com.tr/ASUS-Vivobook-15-X15...,B08H1WX5MY
2,"Lenovo IdeaPad 3 Dizüstü Bilgisayar, Intel Cor...",10.999TL,https://m.media-amazon.com/images/I/61DQ67Cr2H...,https://www.amazon.com.tr/Lenovo-IdeaPad-Bilgi...,B0CQTKWH2M
3,ASUS Vivobook 15 - X1504ZA-NJ566 15.6 inç FHD ...,NaN,https://m.media-amazon.com/images/I/51zyo-VeNS...,https://www.amazon.com.tr/ASUS-Vivobook-15-X15...,B08HLQTYXW
4,Asus Vivobook 15 X1504ZA-NJ180 Intel i5-1235U ...,NaN,https://m.media-amazon.com/images/I/514DRQG0T7...,https://www.amazon.com.tr/Vivobook-X1504ZA-NJ1...,B0DCYYFMYC
...,...,...,...,...,...
397,Game Garaj Slayer3 7XL-4060 V2 C2 Intel Core i...,NaN,https://m.media-amazon.com/images/I/51UwnLziXt...,https://www.amazon.com.tr/Game-Garaj-Slayer3-7...,B0DM5KD12F
398,Acer Nitro V NH.QNCEY.001 Intel Core i5 13420H...,NaN,https://m.media-amazon.com/images/I/61ISCAgKWd...,https://www.amazon.com.tr/Acer-NH-QNCEY-001-Ta...,B0CW9PSJ9F
399,HP 15S-EQ2000NT 4H0J2EA08 Ryzen7 5700U 64GB 1T...,NaN,https://m.media-amazon.com/images/I/81qoseNe+A...,https://www.amazon.com.tr/HP-15S-EQ2000NT-4H0J...,B0DM5SLSJ4
400,HP 255 G9 9M3G7AT02 Ryzen5 5625U 16GB 512SSD 1...,NaN,https://m.media-amazon.com/images/I/81TTHagPRt...,https://www.amazon.com.tr/HP-9M3G7AT02-FreeDOS...,B0D92N5QWS


In [ ]:
# ASIN değerleri kullanilarak ürünlerin review'leri scrap edilecek ve 
# Price değerleri boş olanlar doldurulacak.
# reviewler çekildikten sonra ASIN değerleri datasetten kaldırılıp csv dosyası yeniden extract edilecek.

#TRENDYOL-N11-AMAZON-VATAN BILGISAYAR - HEPSIBURADA (? EĞER OLURSA.)
!pip install requests_html

  Using cached requests_html-0.10.0-py3-none-any.whl (13 kB)
  Using cached pyquery-2.0.1-py3-none-any.whl (22 kB)
  Using cached bs4-0.0.2-py2.py3-none-any.whl (1.2 kB)
  Using cached fake_useragent-2.0.3-py3-none-any.whl (201 kB)
  Using cached pyppeteer-2.0.0-py3-none-any.whl (82 kB)
  Using cached parse-1.20.2-py2.py3-none-any.whl (20 kB)
     ------------------------------------ 101.4/101.4 kB 836.7 kB/s eta 0:00:00
  Using cached pyee-11.1.1-py3-none-any.whl (15 kB)
     -------------------------------------- 166.4/166.4 kB 1.7 MB/s eta 0:00:00
  Using cached cssselect-1.2.0-py2.py3-none-any.whl (18 kB)
  Attempting uninstall: cssselect
    Found existing installation: cssselect 1.1.0
    Uninstalling cssselect-1.1.0:
      Successfully uninstalled cssselect-1.1.0
  Attempting uninstall: certifi
    Found existing installation: certifi 2022.9.14
    Uninstalling certifi-2022.9.14:
      Successfully uninstalled certifi-2022.9.14


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googletrans 4.0.0rc1 requires httpx==0.13.3, but you have httpx 0.24.0 which is incompatible.
conda-repo-cli 1.0.20 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.20 requires nbformat==5.4.0, but you have nbformat 5.5.0 which is incompatible.


In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
import random

class Reviews:
    def __init__(self, asin):
        self.asin = asin
        firefox_options = Options()

        # ✅ Firefox profilini kullan
        firefox_options.add_argument("-profile")
        firefox_options.add_argument("C:/Users/EMREHAN/AppData/Roaming/Mozilla/Firefox/Profiles/xyz123.default-release")  # Doğru yolu gir!

        # ✅ Başlangıçta Headless Modu Kapat (Görünür çalıştır)
        # firefox_options.add_argument("--headless")  # Eğer testten sonra açmak istersen kaldırabilirsin

        # ✅ GeckoDriver'ı başlat
        driver_path = "C:\\Users\\EMREHAN\\PycharmProjects\\geckodriver.exe"
        service = Service(driver_path)
        self.driver = webdriver.Firefox(service=service, options=firefox_options)

        # 📌 Amazon yorum sayfa linki
        self.url = f'https://www.amazon.com.tr/product-reviews/{self.asin}/?pageNumber='

    def pagination(self, page):
        """Belirtilen sayfanın yorumlarını çeker"""
        full_url = self.url + str(page)
        print(f"📌 Sayfa URL'si: {full_url}")  # Hata ayıklama için URL'yi yazdır

        self.driver.get(full_url)

        # ✅ Bot algılanmayı önlemek için mouse hareketi ekleyelim
        actions = ActionChains(self.driver)
        actions.move_by_offset(random.randint(50, 200), random.randint(50, 200)).perform()
        time.sleep(random.uniform(2, 4))  # 🔹 Rastgele bekleme ekle

        # ✅ Sayfayı aşağı kaydırarak yorumların yüklenmesini sağla
        self.driver.execute_script("window.scrollTo(0, document.body.scrollHeight/2);")
        time.sleep(random.uniform(3, 6))  # 🔹 Bekleme süresi ekle

        try:
            # ✅ Yorumların yüklenmesini bekleyelim
            wait = WebDriverWait(self.driver, 15)
            reviews_section = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-hook='review']")))

            # ✅ Yorumları al
            reviews = self.driver.find_elements(By.CSS_SELECTOR, "div[data-hook='review']")
            return reviews if reviews else None  # Eğer yorum yoksa None döndür
        except Exception as e:
            print(f"❌ Hata oluştu: {e}")
            return None

    def parse(self, reviews):
        """Yorumları parse eder ve temizler"""
        total = []
        for review in reviews:
            try:
                title = review.find_element(By.CSS_SELECTOR, 'a[data-hook="review-title"]').text
                rating = review.find_element(By.CSS_SELECTOR, 'i[data-hook="review-star-rating"] span').text
                body = review.find_element(By.CSS_SELECTOR, 'span[data-hook="review-body"] span').text.replace('\n', '').strip()

                data = {
                    'title': title,
                    'rating': rating,
                    'body': body[:100]  # İlk 100 karakter
                }
                total.append(data)
            except:
                continue  # Eğer bir yorumda hata olursa atla
        return total

    def close_driver(self):
        """Tarayıcıyı kapatır"""
        self.driver.quit()


### **🚀 main() FONKSİYONU İLE OTOMATİK ÇALIŞTIRMA**
def main():
    """Amazon ürün yorumlarını çeker ve gösterir"""
    asin = "B08HLQTYXW"  # 📌 Buraya kendi ASIN kodunu yazabilirsin
    amz = Reviews(asin)

    # ✅ İlk 3 sayfanın yorumlarını alalım
    all_reviews = []
    for page in range(1, 4):  # 3 sayfa dolaşır
        print(f"\n📌 Sayfa {page} işleniyor...")
        reviews = amz.pagination(page)
        if reviews:
            parsed_reviews = amz.parse(reviews)
            all_reviews.extend(parsed_reviews)
        else:
            print("✅ Daha fazla yorum bulunamadı.")
            break

    # ✅ Sonuçları göster
    print("\n✅ Tüm Yorumlar:")
    for review in all_reviews:
        print(review)

    # ✅ Tarayıcıyı kapat
    amz.close_driver()

# 📌 Eğer bu dosya direkt çalıştırılırsa, main() fonksiyonunu çalıştır
if __name__ == "__main__":
    main()


SessionNotCreatedException: Message: Failed to set preferences: unknown error


In [ ]:
df.info()

NameError: name 'df' is not defined

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

# specify the target URL
target_url = "https://www.amazon.com/Logitech-G502-Performance-Gaming-Mouse/dp/B07GBZ4Q68/?th=1"

# send a get request to the target url
response = requests.get(target_url)

# check if the response status code is not 200 (OK)
if response.status_code != 200:
    # print an error message with the status code
    print(f"An error occurred with status {response.status_code}")
else:
    # get the page html content
    html_content = response.text

    # parse the html content using BeautifulSoup
    soup = BeautifulSoup(html_content, "html.parser")

    # find all elements with class name "a-profile-name"
    reviewer_names = soup.find_all("span", class_="a-profile-name")
    names_list = [name.text.strip() for name in reviewer_names]

    # find all elements with class name "review-title"
    review_titles = soup.find_all("a", class_="review-title")
    titles_list = [title.text.replace("5.0 out of 5 stars\n", "").strip() for title in review_titles]
  
    # find all elements with class name "review-text-content"
    review_texts = soup.find_all("span", class_="review-text")
    review_texts_list = [text.get_text(separator="\n").strip() for text in review_texts]

    # find all elements with class name "review-date"
    review_dates = soup.find_all("span", class_="review-date")
    review_dates_list = [date.text.strip() for date in review_dates]

    # find all elements with class name "review-rating"
    review_ratings = soup.find_all("i", class_="review-rating")
    review_ratings_list = [rating.text.strip() for rating in review_ratings]

    # find all img elements with class name "review-image-tile"
    review_images = soup.find_all("img", class_="review-image-tile")
    image_urls = [img["src"] for img in review_images]

    # create a dictionary to store the review details
    reviews = {
        "Reviewer Names": names_list,
        "Review Titles": titles_list,
        "Review Texts": review_texts_list,
        "Review Dates": review_dates_list,
        "Review Star Ratings": review_ratings_list,
        "Review Image URLs": image_urls
    }

    # print the dictionary
    print(reviews)

    # specify the CSV file name
    csv_file = "amazon_reviews.csv"
    # open the file in write mode
    with open(csv_file, mode="w", newline="", encoding="utf-8") as file:
        # create a CSV writer object
        writer = csv.writer(file)

        # write the header row
        writer.writerow(["Reviewer Name", "Review Title", "Review Text", "Review Date", "Star Rating", "Image URL"])

        # write the review data
        for i in range(len(names_list)):
            writer.writerow([
                names_list[i],
                titles_list[i] if i < len(titles_list) else "N/A",
                review_texts_list[i] if i < len(review_texts_list) else "N/A",
                review_dates_list[i] if i < len(review_dates_list) else "N/A",
                review_ratings_list[i] if i < len(review_ratings_list) else "N/A",
                image_urls[i] if i < len(image_urls) else "N/A"
            ])

    print(f"Data successfully exported to {csv_file}")


{'Reviewer Names': ['Logitech G502 Hero Gaming Mouse Video', '', '', '', '', '', '', '', '', '', '', 'Amazon Customer', 'Amazon Customer', 'jericho stetson', 'Mike', 'Mike', 'Id10t Waffle', 'Subnet', 'Paola Mercado', 'Elyssa P', 'Valceir', 'Valceir', 'user-MKTSCF', 'Bharg'], 'Review Titles': ['best mouse overall for gaming, work from home, education, or even content creation and more.', 'Nice mouse and good value', 'A Trusted Companion for Yearsâ€”The Ultimate Gaming Mouse', 'Best mouse hands down!'], 'Review Texts': ['this mouse has a main DPI setting that allows 3 DPI presets but also has two buttons to increase DPI or decrease DPI to your own preset on the side allowing 5 extra DPI presets within the main DPI setting. thats 15 DPI settings total for customization. this mouse also has a infinite scroll wheel button that changes the regular scroll your used to into a balanced smooth scroll that speeds up scrolling for actions in games, scrolling pages, or even more. the scrolling is v

In [29]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import InvalidSessionIdException
import pandas as pd
import time
import random

# --------------------------------------------------
# 1. DataFrame ve "Review" Sütunu
# --------------------------------------------------
# Örneğin, daha önceden oluşturulmuş dataset'inizi yükleyin:
# df = pd.read_csv("C:/Users/EMREHAN/Documents/amazon_products.csv")
# Eğer "Review" sütunu yoksa ekleyelim:
if "Review" not in df.columns:
    df["Review"] = ""

# --------------------------------------------------
# 2. Driver Başlatma Fonksiyonu
# --------------------------------------------------
def init_driver():
    chrome_options = Options()
    chrome_options.add_argument("--start-maximized")  # Tam ekran
    chrome_options.add_argument("--disable-popup-blocking")
    chrome_options.add_argument("--user-data-dir=C:/Users/EMREHAN/AppData/Local/Google/Chrome/User Data")
    driver_path = "C:\\Users\\EMREHAN\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe"
    service = Service(driver_path)
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

# İlk driver'ı başlatıyoruz.
driver = init_driver()

# --------------------------------------------------
# 3. Ürünleri İşleme: Yorumları Çekme (Zaten çekilmiş olanları atla)
# --------------------------------------------------
for index, row in df.iterrows():
    # Eğer "Review" sütunu dolu ise, bu ürün zaten işlenmiş demektir; atla.
    if row["Review"]:
        print(f"🔔 Ürün {row['ASIN']} zaten işlenmiş, atlanıyor.")
        continue

    asin = row["ASIN"]
    print(f"\n🔍 {asin} için yorumlar çekiliyor...")
    
    all_reviews = []  # Bu ürünün tüm yorumlarını saklayacağız
    page = 1         # Başlangıç sayfa numarası

    while True:
        review_url = f"https://www.amazon.com.tr/product-reviews/{asin}/?pageNumber={page}"
        print(f"📌 Sayfa {page}: {review_url}")

        try:
            driver.get(review_url)
        except InvalidSessionIdException:
            print("❌ Oturum geçersiz hale geldi, driver yeniden başlatılıyor...")
            driver = init_driver()
            driver.get(review_url)

        # Rastgele bekleme: 5 ile 10 saniye arasında
        time.sleep(random.randint(5, 10))
        
        try:
            wait = WebDriverWait(driver, 15)
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "span[data-hook='review-body']")))
            
            review_elements = driver.find_elements(By.CSS_SELECTOR, "span[data-hook='review-body']")
            review_texts_list = [element.text.strip() for element in review_elements]
            
            # Eğer yorum bulunamadıysa döngüden çık
            if not review_texts_list:
                print(f"✅ {asin} için başka yorum bulunamadı. İşlem tamamlandı!")
                break
            
            all_reviews.extend(review_texts_list)
            
        except Exception as e:
            print(f"❌ Sayfa yüklenirken hata oluştu: {e}")
            break  # Hata durumunda bu ürünün işlenmesini sonlandır
        
        page += 1

    # --------------------------------------------------
    # 4. Çekilen Yorumları DataFrame'e Ekleme
    # --------------------------------------------------
    df.at[index, "Review"] = " | ".join(all_reviews) if all_reviews else "No Reviews"
    
    print("\n📝 Örnek Yorumlar:")
    for example_review in all_reviews[:5]:
        print(f"- {example_review[:200]}...")

# --------------------------------------------------
# 5. Partial Kaydetme İşlemi Kaldırıldı (PermissionError almamak için)
# --------------------------------------------------

# --------------------------------------------------
# 6. İşlem Tamamlandığında Driver'ı Kapat ve Final CSV Kaydet
# --------------------------------------------------
driver.quit()
final_file = "C:/Users/EMREHAN/Documents/amazon_products_updated.csv"
df.to_csv(final_file, index=False, encoding="utf-8")
print(f"\n🎉 Tüm veriler {final_file} dosyasına kaydedildi!")


🔔 Ürün B0CK84VL9X zaten işlenmiş, atlanıyor.
🔔 Ürün B08H1WX5MY zaten işlenmiş, atlanıyor.
🔔 Ürün B0CQTKWH2M zaten işlenmiş, atlanıyor.
🔔 Ürün B08HLQTYXW zaten işlenmiş, atlanıyor.
🔔 Ürün B0DCYYFMYC zaten işlenmiş, atlanıyor.
🔔 Ürün B0CZ31GM8L zaten işlenmiş, atlanıyor.
🔔 Ürün B0CQRKSDYC zaten işlenmiş, atlanıyor.
🔔 Ürün B0DLHMLFZL zaten işlenmiş, atlanıyor.
🔔 Ürün B0DLHDC186 zaten işlenmiş, atlanıyor.
🔔 Ürün B0DLHCCM2J zaten işlenmiş, atlanıyor.
🔔 Ürün B0CZDG6JVX zaten işlenmiş, atlanıyor.

🔍 B0CNKC6H7J için yorumlar çekiliyor...
📌 Sayfa 1: https://www.amazon.com.tr/product-reviews/B0CNKC6H7J/?pageNumber=1
📌 Sayfa 2: https://www.amazon.com.tr/product-reviews/B0CNKC6H7J/?pageNumber=2
📌 Sayfa 3: https://www.amazon.com.tr/product-reviews/B0CNKC6H7J/?pageNumber=3
📌 Sayfa 4: https://www.amazon.com.tr/product-reviews/B0CNKC6H7J/?pageNumber=4
📌 Sayfa 5: https://www.amazon.com.tr/product-reviews/B0CNKC6H7J/?pageNumber=5
❌ Sayfa yüklenirken hata oluştu: Message: 
Stacktrace:
	GetHandleVerifier 

In [24]:
df["Review"]

0      Harika bir bilgisayar, gönderim çok hızlı soru...
1      Günlük kullanıma uygun, şık görünümlü ve perfo...
2      Akademik işlerim için bu ürünü aldım. Fiyat ve...
3      Yıllarca zenbook serisini kullanmış birisi ola...
4      Ürünü bilgisayar öğretmeni arkadaşımın tavsiye...
                             ...                        
397                                                     
398                                                     
399                                                     
400                                                     
401                                                     
Name: Review, Length: 360, dtype: object

In [31]:
df = df[df["Review"] != ""]
print(df)

                                          Product Name     Price  \
0    Lenovo IdeaPad Slim 3 15IAH8 Dizüstü Bilgisaya...  14.999TL   
1    ASUS Vivobook 15 - X1504ZA-NJ1266 15.6 inç FHD...  11.659TL   
2    Lenovo IdeaPad 3 Dizüstü Bilgisayar, Intel Cor...  10.999TL   
3    ASUS Vivobook 15 - X1504ZA-NJ566 15.6 inç FHD ...       NaN   
4    Asus Vivobook 15 X1504ZA-NJ180 Intel i5-1235U ...       NaN   
..                                                 ...       ...   
397  Game Garaj Slayer3 7XL-4060 V2 C2 Intel Core i...       NaN   
398  Acer Nitro V NH.QNCEY.001 Intel Core i5 13420H...       NaN   
399  HP 15S-EQ2000NT 4H0J2EA08 Ryzen7 5700U 64GB 1T...       NaN   
400  HP 255 G9 9M3G7AT02 Ryzen5 5625U 16GB 512SSD 1...       NaN   
401  HP Victus 16-R1077NT Intel Core i5 14500HX 16 ...       NaN   

                                             Image URL  \
0    https://m.media-amazon.com/images/I/61wH5UlX-M...   
1    https://m.media-amazon.com/images/I/51dHk31mrQ...   
2    http

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 360 entries, 0 to 401
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Product Name  360 non-null    object
 1   Price         3 non-null      object
 2   Image URL     360 non-null    object
 3   Link          360 non-null    object
 4   ASIN          360 non-null    object
 5   Review        360 non-null    object
dtypes: object(6)
memory usage: 27.8+ KB


In [35]:
df[df["Review"] == "No Reviews"]

,Product Name,Price,Image URL,Link,ASIN,Review
17,Lenovo IdeaPad Slim 3 Intel N100 4GB 128GB 15....,NaN,https://m.media-amazon.com/images/I/61-b8lqwRy...,https://www.amazon.com.tr/Lenovo-IdeaPad-82XB0...,B0DRD9PDY6,No Reviews
27,Lenovo V15 G4 IRU 83A10096TR i5-13420H 8GB 512...,NaN,https://m.media-amazon.com/images/I/41yHbRLQvf...,https://www.amazon.com.tr/Lenovo-83A10096TR-i5...,B0DHXKNX3F,No Reviews
28,Lenovo Loq 15IRX9 Intel Core I7-13650HX 8gb 1t...,NaN,https://m.media-amazon.com/images/I/617MzluZrd...,https://www.amazon.com.tr/Lenovo-I7-13650HX-Ta...,B0D6NK23BZ,No Reviews
33,Lenovo LOQ 15ARP9 AI AMD Ryzen 7-7435HS 8GB DD...,NaN,https://m.media-amazon.com/images/I/61q7Xm2Mqt...,https://www.amazon.com.tr/Lenovo-7-7435HS-RTX4...,B0DM69TQ8Y,No Reviews
36,Lenovo ThinkPad E14 21M7002PTX Ultra5 125U 16G...,NaN,https://m.media-amazon.com/images/I/71WkKvyj3h...,https://www.amazon.com.tr/Lenovo-ThinkPad-21M7...,B0DNNBCP7R,No Reviews
...,...,...,...,...,...,...
395,CASPER Nirvana C370.4020-4C00B Intel Celeron N...,NaN,https://m.media-amazon.com/images/I/715z789goV...,https://www.amazon.com.tr/Casper-Nirvana-C370-...,B0BGJ7CT5D,No Reviews
396,Dell Vostro 3910MT i7-12700 16GB 512GB SSD Ubu...,NaN,https://m.media-amazon.com/images/I/61MGrPI+Ma...,https://www.amazon.com.tr/Dell-Vostro-3910MT-i...,B0BDMDW19F,No Reviews
397,Game Garaj Slayer3 7XL-4060 V2 C2 Intel Core i...,NaN,https://m.media-amazon.com/images/I/51UwnLziXt...,https://www.amazon.com.tr/Game-Garaj-Slayer3-7...,B0DM5KD12F,No Reviews
399,HP 15S-EQ2000NT 4H0J2EA08 Ryzen7 5700U 64GB 1T...,NaN,https://m.media-amazon.com/images/I/81qoseNe+A...,https://www.amazon.com.tr/HP-15S-EQ2000NT-4H0J...,B0DM5SLSJ4,No Reviews


In [37]:
df["Review"].value_counts()

Review
No Reviews                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [40]:
df = df[df["Review"]!="No Reviews"]
print(df)

                                          Product Name     Price  \
0    Lenovo IdeaPad Slim 3 15IAH8 Dizüstü Bilgisaya...  14.999TL   
1    ASUS Vivobook 15 - X1504ZA-NJ1266 15.6 inç FHD...  11.659TL   
2    Lenovo IdeaPad 3 Dizüstü Bilgisayar, Intel Cor...  10.999TL   
3    ASUS Vivobook 15 - X1504ZA-NJ566 15.6 inç FHD ...       NaN   
4    Asus Vivobook 15 X1504ZA-NJ180 Intel i5-1235U ...       NaN   
..                                                 ...       ...   
383  Asus TUF Gaming A15 FA507NVR-LP005 8GB RTX4060...       NaN   
384  HP Victus 16-S1010NT 9Z790EA 8GB RTX4060 Ryzen...       NaN   
393  Dell G15 5530 i9-13900HX 64 GB 4 TBSSD RTX 406...       NaN   
398  Acer Nitro V NH.QNCEY.001 Intel Core i5 13420H...       NaN   
400  HP 255 G9 9M3G7AT02 Ryzen5 5625U 16GB 512SSD 1...       NaN   

                                             Image URL  \
0    https://m.media-amazon.com/images/I/61wH5UlX-M...   
1    https://m.media-amazon.com/images/I/51dHk31mrQ...   
2    http

In [41]:
df.to_csv("C://Users/Emrehan/Desktop/Emrehan Simsek/veri çekme selenium/datasets/amazon_products.csv", index=False)

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 186 entries, 0 to 400
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Product Name  186 non-null    object
 1   Price         3 non-null      object
 2   Image URL     186 non-null    object
 3   Link          186 non-null    object
 4   ASIN          186 non-null    object
 5   Review        186 non-null    object
dtypes: object(6)
memory usage: 10.2+ KB
